In [1]:
!rm ./GUI_command_line.py

rm: ./GUI_command_line.py: No such file or directory


In [2]:
%%writefile ./GUI_command_line.py
#!/usr/bin/env python
#GUI that retrieves the desired input file
def get_file():
    """
    makes a GUI that retrieves the desired file name for plotting
    Inputs: 
        None
    Returns:
        path_to_file:  str
             path to the desired file
    """
    import PySimpleGUI as sg
    layout = [[sg.InputText('Filename'),sg.FileBrowse()],[sg.OK()]  ]  
    window = sg.Window('Simple data entry window').Layout(layout).Finalize()
    button, values = window.Read()
    window.Close()
    path_to_file=values[0]
    return path_to_file

# make a matplotlib figure
def make_topo_map(path_to_file):
    """ Open path_to_file in a pandas DataFrame, and make a topographic map.
        Inputs: 
            path_to_file: str
                the path to the file with longitude, latitude, elevation
        Outputs:
            fig: matplotlib figure object
    """
    import pandas as pd
    import cartopy.crs as ccrs
    import matplotlib.pyplot as plt
    import numpy as np
    data=pd.read_csv(path_to_file,header=None,delim_whitespace=True)
    data.columns=['lon','lat','elev']
    x=data.lon.unique()
    y=data.lat.unique()
    z=data.elev.values
    Z=np.reshape(z,newshape=(len(y),len(x)))

    proj = ccrs.LambertConformal(central_longitude=x.mean(), central_latitude=y.mean())
    fig = plt.figure(figsize=(6,6), frameon=True) # you need this frameon to be true
    ax = plt.axes(projection=proj)
    plt.pcolormesh(x,y,Z,cmap='RdBu_r',shading='gouraud',transform=ccrs.PlateCarree())
    return fig

## put a figure onto a canvas
def draw_figure(canvas, figure, loc=(0, 0)):
    """ Draw a matplotlib figure onto a Tk canvas

    loc: location of top-left corner of figure on canvas in pixels.

    Inspired by matplotlib source: lib/matplotlib/backends/backend_tkagg.py
    """
    from matplotlib.backends.backend_tkagg import FigureCanvasAgg
    import matplotlib.backends.tkagg as tkagg
    import tkinter as Tk

    # make a Tkagg canvas
    figure_canvas_agg = FigureCanvasAgg(figure)
    figure_canvas_agg.draw()
    # get some dimensions for our figure's bounding box (bbox)
    figure_x, figure_y, figure_w, figure_h = figure.bbox.bounds
    figure_w, figure_h = int(figure_w), int(figure_h)
    # make a "photo" object with correct dimensions out of the canvas
    photo = Tk.PhotoImage(master=canvas, width=figure_w, height=figure_h)
    # put the figure on the photo
    canvas.create_image(loc[0] + figure_w/2, loc[1] + figure_h/2, image=photo)
    tkagg.blit(photo, figure_canvas_agg.get_renderer()._renderer, colormode=2)
    # return the photo object
    return photo


def window_maker(fig,title="Embedded Matplotlib Figure"):
    """ Makes a GUI window to put a matplotlib figure in"""
    import PySimpleGUI as sg

    # get the dimensions of the figure
    figure_x, figure_y, figure_w, figure_h = fig.bbox.bounds
    # define the window layout with a title, a canvas and an OK button
    layout = [[sg.Text(title, font='Any 18')],
          [sg.Canvas(size=(figure_w, figure_h), key='canvas')],
          [sg.OK(pad=((figure_w / 2, 0), 3), size=(4, 2))]]

    # create the window and show it without the plot
    window = sg.Window('Figure in a Window', force_toplevel=True).Layout(layout)
    window.Finalize() # needed to access the canvas element prior to reading the window
    
    # get the canvas element from the window
    canvas=window.FindElement('canvas').TKCanvas
        # show it all again and retrieve buttons
    # add the plot to the canvas
    fig_photo = draw_figure(canvas, fig)
    event, values = window.Read()
    window.Close()


def main():
    # get the file name
    path_to_file=get_file()
    # read in the data and make the mesh
    # make the map
    fig=make_topo_map(path_to_file)
    # make the window
    window_maker(fig,title=path_to_file)


    
# make it so

main()


Writing ./GUI_command_line.py


In [3]:
! chmod a+x GUI_command_line.py